# Chaines de Markov et épidémiologie :
## propagation d’une épidémie dans une population

### Packages :

In [114]:
# Les packages random et matplotlib sont conseillés.
import numpy as np
import matplotlib.pyplot as plt
import random
from copy import copy, deepcopy
from collections import Counter

## 1 Description du modèle

#### 1. Créez la matrice de transition A, la matrice contenant les probabilités de transition entre les différents états. Vérifiez que cette matrice est stochastique

In [11]:
# A la matrice de transition 
# P(2,2) = 1 ; P(0,1) = 0.08 ; P(0,0) = 0.92 ; P(1,1) = 0.93 ; P(1,2) = 0.07 ; P(0,2) = P(1,0) = P(2,0) = P(2,1) = 0
# Avec: P -> la proba ; 0 -> l'etat S ; 1 -> l'etat I ; 2 -> l'etat R
A = np.array([[0.92,0.08,0],[0,0.93,0.07],[0,0,1]])
print(A)


[[0.92 0.08 0.  ]
 [0.   0.93 0.07]
 [0.   0.   1.  ]]


0.92 + 0.08 + 0 = 0 + 0.93 + 0.07 = 0 + 0 + 1 = 1 

Alors la somme de chaque ligne vaut 1

Donc cette matrice est stochastique

#### 2. Créez Π0 la distribution de probabilité initiale.

In [12]:
P_s = 0.9 # Un individu a une probabilité de 0.9 d’être sain
P_i = 0.1 # 0.1 d’être infecté
P_r = 0 # Alors une probabilité de 0 qu'il soit guéri
pi0 = [P_s, P_i, P_r] # Distribution de la probabilité initale
print(pi0)

[0.9, 0.1, 0]


### Tirage aléatoire des états

In [68]:
def initialStatePicker(p, dist_proba_init):
    """
    Fonction implémenter pour l'utilisation en generateRandomSequence, 
    elle trouve l'état initial choisi a travers de la probabilité aléatoire et la distribution initiale des probabilités.
    
    *les parametres
     p : la probabilité aléatoire ; int.
     dist_proba_init : la distribition de la probablité initiale; tab[float] .
     
    *le return : 
     Retourne la valeur clé de l'état.
    
    """
    
    # Nous choisissons un état initial aléatoire en tenant compte de la distribution de la probabilité initiale [0.9, 0.1, 0]
    # c'est a dire l'etat initial peut etre soit un S avec une proba de 90% soit un I avec une proba de 10% 
    # si nous trouvons 0 c'est a dire R avec une proba nul nous recherchons du nouveau un nouveau 
    # état initial qui est différent de R .
    
    if p == 0 : # 0%
        i = dist_proba_init.index(0) # R 
    elif (p > 0) and (p <= 10) : # 10%
        i = dist_proba_init.index(0.1) # I 
    elif (p > 10) and (p <= 100) : # 90%
        i = dist_proba_init.index(0.9) # S 
        
    return i

In [141]:
#precendent -> transition proba / reboucle -> etat suivant



def nextStatePicker(pre, p, matrice_transition):
    """
    Fonction implémenter pour l'utilisation en GenerateRandomSequence,
    elle trouve l'etat suivant a partir de : l'etat precedent, la probabilité aléatoire et la matrice des transitions.
    
    *les parametres:
     pre : l'etat precedent ; int .
     p : la probabilité aléatoire ; int.
     matrice_transition : la matrice contenant les probabilités de transition entre les différents états; tab[float][float] .
     
     *le return:
      Retourne la valeur clé de l'etat suivant.
      
    """
    matrix = deepcopy(matrice_transition) # copie temporaire
    list_AZ = [s for s in matrix[pre]] #nous prenons la liste avec l'indice du precedent 
    list_SZ = [s for s in matrix[pre] if s != 0] # la liste de la matrice avec l'indice du precedent sans les 0 .
    repetitions = Counter(list_SZ) #un dictionnaire où nous mettons les valeurs de la liste avec leurs repetitions 
    # Exemple : [0.5 0.5 0] -> {0.5 : 2 , 0 : 1}
    # Nous applicons ca sur la liste sans les zeros pour des raisons de facilité de calcul et d'optimisation

    minimum = min(list_SZ) # Le minimum de la liste sans les zeros 
                            # afin que nous prenons pas les 0 mais la valeur qui est juste supérieur 
    
    
    list_min = [cpt for cpt, n in enumerate(list_AZ) if n == minimum] # Nous mettons toutes ses positions dans une liste 

    if (p > 0) and (p <= (minimum * 100)) : # Exemple [0.91 0.08 0.01] P devra etre entre 0 et 0.01*100=10%
        if repetitions[minimum] > 1 : # Si la valeur existe plus qu'une fois dans la liste
            index = list_min[0] # Nous prenons la premiere position vu que c'est la premiere fois que nous voyons la valeur
        else :    
            index = list_AZ.index(minimum) # Sinon nous prenons directement l'indice de la valeur vu que la fonction index
                                            # prend que la premiere position de la valeur dans la liste
        return index 
        
    list_SZ.remove(minimum) # Il faut enlever la valeur de la liste sans les 0
    minimum2 = min(list_SZ) # pour trouver le nouveau minimum
    
    
    list_min2 = [cpt for cpt, n in enumerate(list_AZ) if n == minimum2]
    
    if (p > minimum) and (p <= (minimum * 100 + minimum2 * 100)) :
        if repetitions[minimum2] > 1 : 
            if list_min2 == list_min : 
                index = list_min[1] # maintenant si nous voyons la valeur pour la deuxieme fois nous prenons 
                                    # sa deuxième position
            # Et ainsi de suite pour le reste du code de cette fonction, le reste suit le meme logique .
            else :
                index = list_min2[0]   
        else :
            index = list_AZ.index(minimum2)
        return index
        
    list_SZ.remove(minimum2)
    reste = list_SZ[0]
    
    list_reste = [cpt for cpt, n in enumerate(list_AZ) if n == reste]
    
    if (p > (minimum * 100 + minimum2 * 100)) and (p <= 100) :
        if repetitions[reste] > 1 : 
            if list_reste == list_min2 :
                if  list_min2 != list_min : 
                    index = list_min2[1]
                else :
                    index = list_min2[2]
            elif liste_reste == liste_min :
                if liste_min != liste_min2 :
                    index = liste_min[1]
                else :
                    index = liste_min[2]
        else :
            index = list_AZ.index(reste)
    
    return index

In [151]:
def generateRandomSequence(T, dist_proba_init, matrice_transition):
    """
    génére une séquence aléatoire de taille T en utilisant cette chaîne de Markov.
    
    *les parametres:
     T : la longeur de la chaine générer; int .
     dist_proba_init : la distribition de la probablité initiale; tab[float] .
     matrice_transition : la matrice contenant les probabilités de transition entre les différents états; tab[float][float] .
     
    *le return :
     Retourne une sequence genere aleatoirement de taille T; str .
    
    """
    
    #SIR : dict[int:str] 
    SIR = {0 : "S", 1 : "I", 2 : "R"} 
    
    #random_sequence : str  
    random_sequence = "" # Initalisation de la sequence vide, nous allons concatener dedans la sequence a retourner .
    
    # Nous choisissons un état initial aléatoire en tenant compte de la distribution de la probabilité initiale [0.9, 0.1, 0]
    # c'est a dire l'etat initial peut etre soit un S avec une proba de 90% soit un I avec une proba de 10% 
    # si nous trouvons 0 c'est a dire R avec une proba nul nous recherchons du nouveau un nouveau 
    # état initial qui est différent de R .
    proba = np.random.randint(0,100)
    while proba == 0 : # 0%
        proba = np.random.randint(0,100)
    i = initialStatePicker(proba,dist_proba_init)
    
    # Nous insérons le nom de l'etat initial trouvé dans le dictionnaire DIR a partin de sa clé dans
    # le début de la séquence aléatoire. Et nous stockons cet clé dans une variable "precedent" pour pouvoir l'accéder après .
    random_sequence = random_sequence + SIR.get(i) 
    precedent = i # Après, "precedent" devient figurativement le état_i-1 (état precedent) . 
    
    # La boucle dans laquelle nous trouvons les états aléatoirement en tenant compte des probabilités
    # de la matrice des transitions .
    for j in range(1,T):
        # Quand l'indice [precedent,i] = 0 sa probabilité dans la matrice des transitions et nulle
        # c'est a dire la probabilité de i sachant precedent vaut 0, donc la transition qui relie les deux états precedent et i 
        # est inexistante, ce qui nous ramène a rechercher alèatoirement de nouveau un autre état.
        proba = np.random.randint(0,100)
        while (proba == 0) : 
            proba = np.random.randint(0,100)
            
        i = nextStatePicker(precedent,proba,matrice_transition)
   
        precedent = i 
        random_sequence = random_sequence + SIR.get(i) 
        
    return random_sequence

# T = 50
print(generateRandomSequence(50, pi0, A))


SSSSSIIIIIIIIIIIIIIIIIIRRRRRRRRRRRRRRRRRRRRRRRRRRR
